# Evaluation
Click the button below to run this notebook on Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/relmonta/loss-bench/blob/main/evaluation/evaluate_pr.ipynb)

In [ ]:
# If you are using google colab clone the github repository
try:
    %cd /content/
    import google.colab
    ! git clone https://github.com/relmonta/loss-bench.git
    %cd loss-bench
except:
    pass
# Assuming you are in the root directory of the repository
!pip install -r requirements.txt

In [ ]:
try:
    import google.colab
    %cd /content/loss-bench
except:
    # Assuming you are in the root directory of the repository
    pass
import os
import yaml
import pandas as pd
from evaluation.evaluate_pr_utils import *
from evaluation.utils import plot_qq, plot_distributions, plot_event_full, plot_event_zoom, plot_power_spectra
from evaluation.evaluator import Evaluator

In [ ]:
if torch.cuda.is_available():
    print("You are good to go !")
else:
    print("You are not using a GPU. Please check your Google Colab execution setup")

In [ ]:
class EvaluatorPr(Evaluator):
    def __init__(self, experiments, config_path, var_name, comparison_title,
                 metrics=None, start_date="2020-01-01", end_date="2020-12-31", start_date_plot=None, end_date_plot=None, overwrite=False, figformat="png"):
        super().__init__(experiments, config_path, var_name, comparison_title,
                         metrics, start_date, end_date, start_date_plot, end_date_plot, overwrite, figformat)

    def save_metrics_summary(self):
        # Draw temporal evolution, spectrum, etc.
        # Dataframe with metrics for each experiment
        self.df = pd.DataFrame.from_dict(self.results, orient='index')
        self.df.index.name = 'Model'
        self.df.to_csv(os.path.join(self.plot_path,
                  'metrics_summary.csv'), float_format='%.3f')
        print(
            f"Saved {os.path.join(self.plot_path, 'metrics_summary.csv')}")

    def plot_temporal_evolution(self):
        plot_temporal_evolution(self.plot_path, self.y_preds, self.y_true,
                                self.experiments, self.exp_names, self.common_data['dates'], self.start_date_plot, self.end_date_plot, self.colors_dict, figformat=self.figformat)
    
    def plot_power_spectra(self):
        plot_power_spectra(self.plot_path, self.y_preds, self.y_true,
                                               self.experiments, self.exp_names, self.line_styles, self.colors_dict, figformat=self.figformat)

    def plot_event_zoom(self):
        plot_event_zoom(self.plot_path, self.y_preds, self.y_true,
                        self.experiments, self.exp_names, self.common_data, variable="pr", title="Precipitation", label=r"Precipitation [mm.day$^{-1}$]", figformat=self.figformat, zoom_extent=[-6, 24, 35, 52], sel_criteria=["2020-03-02"])

    def plot_event_full(self):
        plot_event_full(self.plot_path, self.y_preds, self.y_true,
                        self.experiments, self.exp_names, self.common_data, variable="pr", title="Precipitation", label=r"Precipitation [mm.day$^{-1}$]", figformat=self.figformat, sel_criteria=["2020-02-11"])

    def plot_qq(self):
        plot_qq(self.plot_path, self.y_preds, self.y_true,
                self.experiments, self.exp_names, self.colors_dict, threshold=1, scales=["log"], var_title="precipitation", unit=r"[mm.day$^{-1}$]", figformat=self.figformat)

    def plot_rx1day(self):
        plot_rx1day(self.plot_path, self.df,
                    self.common_data['rx1day_true'], self.exp_names, self.colors_dict, figformat=self.figformat)

    def plot_distributions_max(self):
        plot_distributions_max(self.plot_path, self.y_preds, self.y_true,
                                                self.experiments, self.exp_names, self.colors_dict, figformat=self.figformat)

    def plot_distributions(self):
        plot_distributions(self.plot_path, self.y_preds, self.y_true,
                           self.experiments, self.exp_names, self.results, self.colors_dict, var_title=r"Precipitation", unit=r"mm.day$^{-1}$", emd_unit="mm", log_scale=True, figformat=self.figformat)

    def plot_violin_max(self):
        plot_violin_max(self.plot_path, self.y_preds, self.y_true,
                        self.experiments, self.exp_names, self.colors_dict, figformat=self.figformat)

    def plot_div_plot(self):
        plot_div_plot(self.plot_path, self.y_preds, self.y_true,
                                   self.experiments, self.exp_names, estimation_threshold=10, figformat=self.figformat)


### Setup

In [ ]:
import os, yaml

# Load comparison config
with open('evaluation/compare_pr.yaml', 'r') as f:
    config = yaml.safe_load(f)

var_name = config['variable']

### Run evaluator for one comparison

In [ ]:
comparison_title, experiments = list(config['comparisons'].items())[0]

evaluator = EvaluatorPr(
    experiments,
    os.path.join('configs', f'exp_config_{var_name}.yaml'),
    var_name,
    comparison_title,
    **config["evaluator_kwargs"]
)
evaluator.save_metrics_summary()


### Radar chart

In [ ]:
display(evaluator.df)

In [ ]:
evaluator.plot_radar_chart(title=None, higher_is_better=True)

### Generate plots

In [ ]:
evaluator.plot_temporal_evolution()


In [ ]:

evaluator.plot_qq()


In [ ]:

evaluator.plot_rx1day()


In [ ]:

evaluator.plot_div_plot()

In [ ]:

evaluator.plot_distributions()


In [ ]:

evaluator.plot_distributions_max()


In [ ]:

evaluator.plot_power_spectra()


In [ ]:

evaluator.plot_event_zoom()


In [ ]:

evaluator.plot_event_full()